In [1]:
from tensorflow.keras.models import load_model
#from tensorflow.python.keras.engine import base_layer_v1
import cv2
import numpy as np
from random import choice
import time
from datetime import datetime
import tensorflow_hub as hub


In [2]:
REV_CLASS_MAP = {
    0: "none",
    1: "one",
    2: "two",
    3: "three",
    4: "four",
    5: "five",
    6: "six"
}


In [3]:
def mapper(val):
    return REV_CLASS_MAP[val]

In [4]:
def calculate_sum(move1, move2,sums,out):
    if move1 == move2:
        out=True
        return (out,sums)

    elif move1 == "one":
        sums=sums+1
        return (out,sums)
    elif move1=="two":
        sums=sums+2
        return (out,sums)        
    elif move1=="three":
        sums=sums+3
        return (out,sums)        
    elif move1=="four":
        sums=sums+4
        return (out,sums)    
    elif move1=="five":
        sums=sums+5
        return (out,sums)    
    elif move1=="six":
        sums=sums+6
        return (out,sums)

In [5]:
model = load_model("hand-cricket-model_2.h5")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [6]:
cap = cv2.VideoCapture(0)
prev_move = None
sums=0
target=0
play="Batting"
start="True"
out=False
TIMER = int(8)
prev = time.time()
last_detected = datetime.now()
start=True
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
writer = cv2.VideoWriter('student_capture5.mp4', cv2.VideoWriter_fourcc(*'XVID'),25, (width, height))

while True:
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    
    if not ret:
        continue
    if play !="Computer Won":
        
        # rectangle for user to play
        cv2.rectangle(frame, (410, 80), (620, 300), (255, 255, 255), 2)
        # rectangle for computer to play
        cv2.rectangle(frame, (20, 80), (200, 220), (255, 255, 255), 2)
    

    # extract the region of image within the user rectangle
    roi = frame[80:300, 410:620]
    img = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
    img = cv2.resize(roi, (210, 220))

    # predict the move made
    pred = model.predict(np.array([img]))
    move_code = np.argmax(pred[0])
    user_move_name = mapper(move_code)

    # predict the winner (human vs computer)
    if prev_move != user_move_name:
        if user_move_name != "none":
            computer_move_name = choice(['one', 'two', 'three','four','five','six'])
            if play=="Batting":
                out,sums = calculate_sum(user_move_name, computer_move_name,sums,out)
                print(out)
                if out:
                    #cv2.putText(frame, "Out!",(230, 340), font, 1.0, (191, 252, 85), 2, cv2.LINE_AA)
                    play="Bowling"
                                   
            elif play=="Bowling":
                out,target=calculate_sum(computer_move_name, user_move_name,target,out)
                print(out)
                if target > sums:
                        play="Computer Won"
                if out:
                    if target< sums:
                        play="You Won"
                    else:
                        play="Tie"
        else:
            computer_move_name = "none"
            sums = sums
    prev_move = user_move_name
    
    if play=="Batting" or play=="Bowling":
    # display the information
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(frame, "Computer's Move: " + computer_move_name,
                (20, 50), font, 0.7, (45, 45, 240), 2, cv2.LINE_AA)
        cv2.putText(frame, "Your Move: " + user_move_name,
                (410, 50), font, 0.7, (45, 45, 240), 2, cv2.LINE_AA)
        cv2.putText(frame, "Your Score " + str(sums),
                (410, 330), font, 0.6, (191, 252, 85), 2, cv2.LINE_AA)
        cv2.putText(frame, "Computer's Score: " + str(target),
                (20, 330), font, 0.6, (191, 252, 85), 2, cv2.LINE_AA)
        cv2.putText(frame, str(play),
                (230, 370), font, 1.2, (69, 224, 116), 2, cv2.LINE_AA)
        cv2.putText(frame, "Hand-Cricket",
                (70, 450), cv2.FONT_HERSHEY_COMPLEX_SMALL, 3.0, (12, 247, 247), 2, cv2.LINE_AA)
        if computer_move_name != "none":
            icon = cv2.imread(
            "images/{}.jpg".format(computer_move_name))
            icon = cv2.resize(icon, (200, 220))
            frame[80:300, 20:220] = icon
            
            
    else:
        font = cv2.FONT_HERSHEY_SIMPLEX
        #cv2.putText(frame, str(play),(250, 390), cv2.FONT_HERSHEY_COMPLEX_SMALL, 2.0, (69, 224, 116), 2, cv2.LINE_AA)
        cv2.putText(frame, "Your Score: " + str(sums),(420, 430), font, 0.7, (191, 252, 85), 2, cv2.LINE_AA)
        cv2.putText(frame, "Computer Score: " + str(target),(20, 430), font, 0.7, (191, 252, 85), 2, cv2.LINE_AA)
        if play=="Computer Won":
            if start:
                display_move=computer_move_name
                start=False
            icon = cv2.imread("images/lost_smiley.png")
            icon = cv2.resize(icon, (210, 210))
            frame[140:350, 200:410] = icon
            icon2 = cv2.imread("images/{}.jpg".format(display_move))
            icon2 = cv2.resize(icon2, (100, 100))
            frame[0:100, 250:350] = icon2
            cv2.putText(frame,"You Lost",(200, 390), cv2.FONT_HERSHEY_COMPLEX_SMALL, 2.0, (7, 7, 242), 2, cv2.LINE_AA)
            cv2.putText(frame, "Computer's Move",(200, 120), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1.0, (191, 252, 85), 2, cv2.LINE_AA)
        elif play=="You Won":
            cv2.putText(frame,"You Won",(220, 390), cv2.FONT_HERSHEY_COMPLEX_SMALL, 2.0, (69, 224, 116), 2, cv2.LINE_AA)
            icon = cv2.imread("images/win.jpg".format(computer_move_name))
            icon = cv2.resize(icon, (640, 300))
            frame[40:340, 0:640] = icon
        elif play=="Tie":
            cv2.putText(frame,"Tie",(280, 390), cv2.FONT_HERSHEY_COMPLEX_SMALL, 2.0, (69, 224, 116), 2, cv2.LINE_AA)
            icon = cv2.imread("images/tie.jpg".format(computer_move_name))
            icon = cv2.resize(icon, (640, 300))
            frame[40:340, 0:640] = icon


    if out:
        while TIMER >= 0:
            ret, frame = cap.read()
            frame = cv2.flip(frame, 1)
            if computer_move_name != "none":
                icon = cv2.imread(
                "images/{}.jpg".format(computer_move_name))
                icon = cv2.resize(icon, (100, 100))
                frame[0:100, 250:350] = icon
            cv2.putText(frame, "Computer's Move",(200, 120), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1.0, (191, 252, 85), 2, cv2.LINE_AA)
            icon2 = cv2.imread("images/out.jpg")
            icon2 = cv2.resize(icon2, (510, 300))
            frame[150:450, 50:560] = icon2
            cv2.putText(frame, "OUT!",(250, 390), cv2.FONT_HERSHEY_COMPLEX_SMALL, 2.0, (0, 240, 255), 3, cv2.LINE_AA)
            cv2.imshow("Hand-Cricket", frame)
            w=cv2.waitKey(125)
            if w == ord('q'):
                break
            cur = time.time()
            writer.write(frame)
            if cur-prev >= 1:
                prev = cur
                TIMER = TIMER-1
        out=False
        TIMER=int(8)
    else:
        cv2.imshow("Hand-Cricket", frame)
    
    cv2.imshow("Test",img)
    writer.write(frame)

    k = cv2.waitKey(10)
    if k == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
writer.release()


False
False
False
False
False
False
False
False
False
False
False
True
True
